## 1. Understanding the Data

#### Import the Data, check for NaNs, Datatypes etc.

In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install numpy

In [ ]:
pip instal sklearn

In [ ]:
pip install openpyxl

In [2]:
import pandas as pd
import numpy as np
# Import Data from Excel
# Display Data

In [2]:
#check data types

In [4]:
# check for Nan 

ProductionDate             0
RegistrationDate           0
Country                    0
CountryNr                  0
BM                         0
VehicleClassification      0
Type                     261
Code                       0
status                     0
Buffer                     0
dtype: int64

In [4]:
 #drop Nan Rows

### Insights:


## 2. Data Preparation

#### Split Code Column

In [2]:
#create dummies for code-features (1 - code is in vehicle / 0 - code is not in vehicle)
#Split all code into features. If a code is availabe set to 1

# first create a set of all codes (a set does not contain any dublicate) => split Code column by ','

# Fill empty fields with a 0. If a code is not availabe in a vehicle you can set it to 0



In [11]:
#convert columns into integer


#### Convert Dates to Datetime and Calculate Buffer

In [13]:
#Convert Dates to Datetime ( Year - month - day)


In [8]:
# Calclulate Buffer as RegistrationDate - ProductionDate. Save Calculation as a new column "Buffer"


#### Convert ProductionDate into Catagories

In [18]:
# turn ProductionDate into Year, Month, Day

#### Create Dummies for catagorical features

In [19]:
# convert catagorical into single features

In [3]:
df=pd.read_excel('TrainingsData.xlsx')
df=df.astype({'RegistrationDate': 'Int32'})
#drop the vehicles in training data
df = df[(df.RegistrationDate.apply(lambda row: len(str(row))) != 6 )]
codes_df = df.apply(lambda x: pd.Series(1, index = list(set(x['Code'].split(',')))), axis=1)
# Fill empty fields with a 0. If a code is not availabe in a vehicle you can set it to 0
codes_df.fillna(0, inplace=True)
#Convert Dates to Datetime
df.ProductionDate = pd.to_datetime(df.ProductionDate, format='%Y%m%d', errors='coerce')
df.RegistrationDate = pd.to_datetime(df.RegistrationDate, format='%Y%m%d', errors='coerce')
df['status']=~pd.isna(df.RegistrationDate) # Flag rows with a status
# Calclulate Buffer as RegistrationDate - ProductionDate
df.loc[df['status']==True,'Buffer'] = (df.RegistrationDate-df.ProductionDate).dt.days
latest_registration = max(df.RegistrationDate)
df.loc[df['status']==False,'Buffer'] = (latest_registration-df.ProductionDate).dt.days
# delete negative Buffers
df = df[df.Buffer>=0]
# combine both dataframes 
df_final=pd.concat([df,codes_df], axis=1)
df_final=df_final.drop(columns=['RegistrationDate','Code','']) # drop RegistrationDate and Code ( is splitted)
df_final=df_final.astype({'status': 'boolean'}) 
# turn ProductionDate into Year, Month, Day
df_final['Year']=df_final.ProductionDate.apply(lambda rows: str(rows.year))
df_final['Month']=df_final.ProductionDate.apply(lambda rows: str(rows.month))
df_final['Day']=df_final.ProductionDate.apply(lambda rows: str(rows.day))
# convert catagorical into single features
cat_col=df_final.select_dtypes(include=['object']).columns # Select all catagorical features
df_final=pd.get_dummies(df_final,prefix=cat_col, prefix_sep='_', drop_first=True) # create dummies
df_final.drop(columns=['ProductionDate'],inplace=True) # Drop ProductionDate features

In [5]:
df_final.to_csv("TrainingData2.csv.zip", 
           index=False, 
           compression="zip")

In [7]:
df

SyntaxError: invalid syntax (<ipython-input-7-737f08a1bdec>, line 1)

#### Create finale df

In [20]:
# Prepare X,y


## Modeling

In [113]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

#Split Data


In [114]:
#Instance Model RandomForestRegressor n_estimators=100,max_depth=30,random_state=0

#Predict

#Score


## 5. Evaluation

In [115]:
#Print training and testing score
print("The rsquared on the training data was {}.  The rsquared on the test data was {}.".format(train_score, test_score))

The rsquared on the training data was 0.8613265602720486.  The rsquared on the test data was 0.5848184866847197.


In [118]:
avarage_buffer_test=y_test_preds.mean()
avarage_buffer_train=y_train_preds.mean()
print("The avarage buffer on the training data was {}.  The avarage buffer on the test data was {}.".format(round(avarage_buffer_train,2), round(avarage_buffer_test,2)))

The avarage buffer on the training data was 91.32.  The avarage buffer on the test data was 92.59.


### Question Nr. 5: How much better is an ML algorithm compared to expert prediction?

In [120]:
#Print training and testing score
print("The MAE on the training data was {}.  The MAE on the test data was {}.".format(train_mae, test_mae))

The MAE on the training data was 22.670151107527765.  The MAE on the test data was 38.24912806286127.


### => Compared to the 70.19 MAE on the expert excel sheet the MAE was on the test data 39.25. That is a 45% better prediction! 